## getting patients with stroke diagnostics 

In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
def all_upper(my_list):
    return [x.upper() for x in my_list]
#Admissions
admission_cols=["subject_id", "hadm_id",  "deathtime","admittime","dischtime", "diagnosis", 
      "hospital_expire_flag" ,"has_chartevents_data"]
admissions_df=pd.read_csv("./tables/ADMISSIONS.csv",usecols=all_upper(admission_cols))
#keep only admissions where we have charts data
admissons_df=admissions_df[admissions_df['HAS_CHARTEVENTS_DATA']==1]
#dropping has charts events
admissions_df.drop("HAS_CHARTEVENTS_DATA",axis=1,inplace=True)

#Diagnoses_ICD
diag_cols=["subject_id"  ,"hadm_id" ,"icd9_code" ]
diagnoses_df=pd.read_csv('./tables/DIAGNOSES_ICD.csv',usecols=all_upper(diag_cols))
diagnoses_df

#D_ICD_Diagnoses
d_icd_cols=["icd9_code" ,"long_title" ]
d_icd_df=pd.read_csv("./tables/D_ICD_DIAGNOSES.csv",usecols=all_upper(d_icd_cols))
d_icd_df

merged_icd = pd.merge(diagnoses_df ,d_icd_df, on='ICD9_CODE')

In [2]:
d_icd_df

,ICD9_CODE,LONG_TITLE
0,01166,"Tuberculous pneumonia [any form], tubercle bac..."
1,01170,"Tuberculous pneumothorax, unspecified"
2,01171,"Tuberculous pneumothorax, bacteriological or h..."
3,01172,"Tuberculous pneumothorax, bacteriological or h..."
4,01173,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...
14562,V7399,Special screening examination for unspecified ...
14563,V740,Screening examination for cholera
14564,V741,Screening examination for pulmonary tuberculosis
14565,V742,Screening examination for leprosy (Hansen's di...


## extracting by icd9 codes

In [231]:
import re

strings = ["4301", "1234", "431A", "432B", "9999", "436XYZ"]

pattern = r'^(430|431|432|433|434|436)'

res = [code for code in code_list if re.match(pattern, code)]

print(res)


['430', '431', '4320', '4321', '4329', '43300', '43301', '43310', '43311', '43320', '43321', '43330', '43331', '43380', '43381', '43390', '43391', '43400', '43401', '43410', '43411', '43490', '43491', '436']


In [3]:
matching_codes=[]
for code in d_icd_df['ICD9_CODE'].unique():
    if code.startswith(('430', '431','432', '433','434','436')):
        matching_codes.append(code)
# import re        
# pattern = r'^(430|431|432|433|434|436)'
# res = [code for code in code_list if re.match(pattern, code)]

# print(res)


In [4]:
len(matching_codes)

24

In [5]:
d_icd_df[d_icd_df['ICD9_CODE'].isin(matching_codes)]

,ICD9_CODE,LONG_TITLE
4501,430,Subarachnoid hemorrhage
4502,431,Intracerebral hemorrhage
4503,4320,Nontraumatic extradural hemorrhage
4504,4321,Subdural hemorrhage
4505,4329,Unspecified intracranial hemorrhage
4506,43300,Occlusion and stenosis of basilar artery witho...
4507,43301,Occlusion and stenosis of basilar artery with ...
4508,43310,Occlusion and stenosis of carotid artery witho...
4509,43311,Occlusion and stenosis of carotid artery with ...
4510,43320,Occlusion and stenosis of vertebral artery wit...


In [8]:
merged_icd[merged_icd['ICD9_CODE'].isin(matching_codes)]

,SUBJECT_ID,HADM_ID,ICD9_CODE,LONG_TITLE
227528,124,172461,43331,Occlusion and stenosis of multiple and bilater...
227529,1854,182779,43331,Occlusion and stenosis of multiple and bilater...
227530,3987,106870,43331,Occlusion and stenosis of multiple and bilater...
227531,6620,140314,43331,Occlusion and stenosis of multiple and bilater...
227532,9471,129609,43331,Occlusion and stenosis of multiple and bilater...
...,...,...,...,...
631656,31169,185065,43400,Cerebral thrombosis without mention of cerebra...
631657,70807,129144,43400,Cerebral thrombosis without mention of cerebra...
632661,32045,148243,43381,Occlusion and stenosis of other specified prec...
632662,41768,170888,43381,Occlusion and stenosis of other specified prec...


In [6]:
print('number of unique patients: ',merged_icd[merged_icd['ICD9_CODE'].isin(matching_codes)]['HADM_ID'].nunique())

number of unique patients:  4684


- Stroke/other cerebrovascular disease 430-438
- mostly commonly used icd9 codes for stroke per the systemic review "major adverserial.." 430-435 and 436
- 437-438 Late effects and other cerebrovascular disease

In [7]:
stroke_patients=merged_icd[merged_icd['ICD9_CODE'].isin(matching_codes)]['HADM_ID'].unique()

In [8]:
stroke_admissions=admissions_df[admissions_df['HADM_ID'].isin(stroke_patients)]
stroke_admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG
29,360,154871,2176-06-25 17:50:00,2176-07-02 14:28:00,NaN,CEREBROVASCULAR ACCIDENT;TELEMETRY,0
56,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,NaN,CAROTID STENOSIS\CAROTID ANGIOGRAM,0
70,67,155252,2157-12-02 00:45:00,2157-12-02 03:55:00,2157-12-02 03:55:00,SUBARACHNOID HEMORRHAGE,1
84,81,175016,2192-01-09 18:50:00,2192-01-11 13:00:00,NaN,INTRACRANIAL HEMORRHAGE;TELEMETRY,0
86,83,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,CAROTID STENOSIS,0
...,...,...,...,...,...,...,...
58905,95404,134147,2166-01-14 22:30:00,2166-02-14 16:40:00,NaN,SUBARACHNOID HEMORRHAGE,0
58917,98642,162213,2179-02-20 14:21:00,2179-03-19 18:51:00,NaN,ACUTE CORONARY SYNDROME,0
58927,98669,108710,2147-04-07 15:09:00,2147-04-14 10:00:00,2147-04-14 10:00:00,SUBARACHNOID HEMORRHAGE,1
58954,98748,122488,2166-12-31 08:00:00,2167-01-05 12:40:00,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0


In [9]:
#merging with patients table
#Patients
patient_cols=["DOB" ,"DOD" ,"Gender" ,"subject_id"]
patients_df=pd.read_csv("./tables/PATIENTS.csv",usecols=all_upper(patient_cols))
stroke_admissions=stroke_admissions.merge(patients_df,on="SUBJECT_ID")
stroke_admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,DOB,DOD
0,360,154871,2176-06-25 17:50:00,2176-07-02 14:28:00,NaN,CEREBROVASCULAR ACCIDENT;TELEMETRY,0,F,2100-02-05 00:00:00,NaN
1,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,NaN,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,F,2072-02-04 00:00:00,NaN
2,67,155252,2157-12-02 00:45:00,2157-12-02 03:55:00,2157-12-02 03:55:00,SUBARACHNOID HEMORRHAGE,1,M,2084-06-05 00:00:00,2157-12-02 00:00:00
3,81,175016,2192-01-09 18:50:00,2192-01-11 13:00:00,NaN,INTRACRANIAL HEMORRHAGE;TELEMETRY,0,M,2106-12-20 00:00:00,2192-01-12 00:00:00
4,83,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,CAROTID STENOSIS,0,F,2076-09-15 00:00:00,2142-11-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...
4679,95404,134147,2166-01-14 22:30:00,2166-02-14 16:40:00,NaN,SUBARACHNOID HEMORRHAGE,0,F,2129-07-16 00:00:00,NaN
4680,98642,162213,2179-02-20 14:21:00,2179-03-19 18:51:00,NaN,ACUTE CORONARY SYNDROME,0,M,2106-03-04 00:00:00,2179-05-04 00:00:00
4681,98669,108710,2147-04-07 15:09:00,2147-04-14 10:00:00,2147-04-14 10:00:00,SUBARACHNOID HEMORRHAGE,1,F,2072-07-04 00:00:00,2147-04-14 00:00:00
4682,98748,122488,2166-12-31 08:00:00,2167-01-05 12:40:00,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,2084-10-30 00:00:00,NaN


In [10]:
# code snipet to convert time columns in dataframe to datetime type
time_cols= [x for x in stroke_admissions.columns if "TIME" in x or x=="DOB" or x=="DOD"]
stroke_admissions[time_cols]= stroke_admissions[time_cols].apply(pd.to_datetime)
stroke_admissions.dtypes
stroke_admissions['AGE'] = (stroke_admissions['ADMITTIME'].dt.year - stroke_admissions['DOB'].dt.year).astype(int)
stroke_admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,DOB,DOD,AGE
0,360,154871,2176-06-25 17:50:00,2176-07-02 14:28:00,NaT,CEREBROVASCULAR ACCIDENT;TELEMETRY,0,F,2100-02-05,NaT,76
1,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,NaT,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,F,2072-02-04,NaT,64
2,67,155252,2157-12-02 00:45:00,2157-12-02 03:55:00,2157-12-02 03:55:00,SUBARACHNOID HEMORRHAGE,1,M,2084-06-05,2157-12-02,73
3,81,175016,2192-01-09 18:50:00,2192-01-11 13:00:00,NaT,INTRACRANIAL HEMORRHAGE;TELEMETRY,0,M,2106-12-20,2192-01-12,86
4,83,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaT,CAROTID STENOSIS,0,F,2076-09-15,2142-11-20,66
...,...,...,...,...,...,...,...,...,...,...,...
4679,95404,134147,2166-01-14 22:30:00,2166-02-14 16:40:00,NaT,SUBARACHNOID HEMORRHAGE,0,F,2129-07-16,NaT,37
4680,98642,162213,2179-02-20 14:21:00,2179-03-19 18:51:00,NaT,ACUTE CORONARY SYNDROME,0,M,2106-03-04,2179-05-04,73
4681,98669,108710,2147-04-07 15:09:00,2147-04-14 10:00:00,2147-04-14 10:00:00,SUBARACHNOID HEMORRHAGE,1,F,2072-07-04,2147-04-14,75
4682,98748,122488,2166-12-31 08:00:00,2167-01-05 12:40:00,NaT,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,2084-10-30,NaT,82


In [12]:
stroke_admissions=stroke_admissions[["SUBJECT_ID","HADM_ID","GENDER","AGE","DIAGNOSIS","HOSPITAL_EXPIRE_FLAG"]]
data=stroke_admissions.copy()

In [13]:
#keeping only adults
data=data[data['AGE']>=18]
data

,SUBJECT_ID,HADM_ID,GENDER,AGE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG
0,360,154871,F,76,CEREBROVASCULAR ACCIDENT;TELEMETRY,0
1,55,190665,F,64,CAROTID STENOSIS\CAROTID ANGIOGRAM,0
2,67,155252,M,73,SUBARACHNOID HEMORRHAGE,1
3,81,175016,M,86,INTRACRANIAL HEMORRHAGE;TELEMETRY,0
4,83,158569,F,66,CAROTID STENOSIS,0
...,...,...,...,...,...,...
4679,95404,134147,F,37,SUBARACHNOID HEMORRHAGE,0
4680,98642,162213,M,73,ACUTE CORONARY SYNDROME,0
4681,98669,108710,F,75,SUBARACHNOID HEMORRHAGE,1
4682,98748,122488,M,82,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0


In [13]:
data.to_parquet('stroke_admissions.parquet',index=False)

# Chartevents

a final list of item ids to keep
- SPO2 646
- Heart Rate (211, 220045) 
- Blood glucose 3744	missing
- admission height 920,  height in cm 226730 
- admission weight 226512, weight 3693 (missing)
- Respiratory Rate 220210 618 
- Arterial Blood Pressure systolic 220050
- Arterial Blood Pressure diastolic 220051
- Non Invasive Blood Pressure systolic 220179
- Non Invasive Blood Pressure diastolic 220180
- 220052	Arterial Blood Pressure mean
- 220181	Non Invasive Blood Pressure mean

In [14]:
import pandas as pd
import dask.dataframe as dd
import pyarrow as pa
from dask.diagnostics import ProgressBar

chartevents_csv_filepath = './tables/CHARTEVENTS.csv'
path_to_dir_chartevents_clean = './'
#get a list of admissions 
admissions=data['HADM_ID'].unique()

# Read CHARTEVENTS.csv as Dask DataFrame
# Data types based on MIMIC schema specification https://mit-lcp.github.io/mimic-schema-spy/tables/chartevents.html
# Problem: Complicated use of intger data types with NaNs in Pandas, see https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#nan-integer-na-values-and-na-type-promotions
# Decision: Floats and integers are read in as 'float64', strings as 'object', and timestamps via Dask's parse_dates provided for this purpose.
chartevents = dd.read_csv(chartevents_csv_filepath , parse_dates=['CHARTTIME','STORETIME'], dtype={
    'ROW_ID': 'float64', # int4 according to specification
    'SUBJECT_ID': 'float64', # int4 according to specification
    'HADM_ID': 'float64', # int4 according to specification
    'ICUSTAY_ID': 'float64', # int4 according to specification
    'ITEMID': 'float64', # int4 according to specification
    'CGID': 'float64', # int4 according to specification
    'VALUE': 'object',
    'VALUENUM': 'float64', # float8 according to specification
    'VALUEUOM': 'object',
    'WARNING': 'float64', # int4 according to specification
    'ERROR': 'float64', # int4 according to specification
    'RESULTSTATUS': 'object',
    'STOPPED': 'object'})

# Create list of relevant ITEMIDs to filter by
itemid_filter=[646,211,220045,3744,920,226730,226512,3693,220210,618,220050,220051,220179,220180,220052,220181,223762,676]

with ProgressBar():
    # Filter by ITEMIDs
    chartevents_subset = chartevents[chartevents.ITEMID.isin(itemid_filter)]
    # Drop rows without ICUSTAY_ID
#     chartevents_subset = chartevents_subset.dropna(how='any', subset=['ICUSTAY_ID'])
    # Keep only rows without error, coded by value 0 in ERROR column
#     chartevents_subset = chartevents_subset[chartevents_subset.ERROR.isin([0])]
    #keep only rows with filtered patients
    chartevents_subset=chartevents_subset[chartevents_subset['HADM_ID'].isin(admissions)]
    # Apply previously defined commands to Dask DataFrame, resulting in desired Pandas DataFrame
    chartevents_subset = chartevents_subset.compute()

# Sort rows and reset index (not essential, but provides better overview)
chartevents_subset = chartevents_subset.sort_values(by=['ICUSTAY_ID', 'CHARTTIME','ITEMID']).reset_index(drop=True)
#filter columns
chartevents_subset=chartevents_subset[["SUBJECT_ID","HADM_ID","ICUSTAY_ID","ITEMID","CHARTTIME","VALUE"]]

# Save as parquet file
chartevents_subset.to_parquet(path_to_dir_chartevents_clean+'stroke_charts.parquet', engine='pyarrow')

[########################################] | 100% Completed |  4min 47.9s


In [14]:
path_to_dir_chartevents_clean = './'
chartevents = pd.read_parquet(path_to_dir_chartevents_clean+'stroke_charts.parquet', engine='pyarrow')
chartevents

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,VALUE
0,74032.0,117458.0,200016.0,226512.0,2150-12-02 16:18:00,64
1,74032.0,117458.0,200016.0,220045.0,2150-12-02 16:22:00,67
2,74032.0,117458.0,200016.0,220045.0,2150-12-02 16:23:00,67
3,74032.0,117458.0,200016.0,220210.0,2150-12-02 16:23:00,14
4,74032.0,117458.0,200016.0,220045.0,2150-12-02 16:25:00,70
...,...,...,...,...,...,...
3077334,88683.0,164352.0,NaN,220045.0,2189-10-03 18:00:00,58
3077335,88683.0,164352.0,NaN,220179.0,2189-10-03 18:00:00,156
3077336,88683.0,164352.0,NaN,220180.0,2189-10-03 18:00:00,55
3077337,88683.0,164352.0,NaN,220210.0,2189-10-03 18:00:00,18


In [16]:
chartevents.dtypes

SUBJECT_ID           float64
HADM_ID              float64
ICUSTAY_ID           float64
ITEMID               float64
CHARTTIME     datetime64[ns]
VALUE                 object
dtype: object

In [15]:
#joining with item_id
item_df=pd.read_csv('final_d_items.csv')
chartevents=chartevents.merge(item_df[["ITEMID","LABEL"]],on="ITEMID")
chartevents

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,VALUE,LABEL
0,74032.0,117458.0,200016.0,226512.0,2150-12-02 16:18:00,64,Admission Weight (Kg)
1,56369.0,198650.0,200033.0,226512.0,2198-08-07 19:00:00,74,Admission Weight (Kg)
2,29708.0,175436.0,200038.0,226512.0,2143-10-25 01:41:00,72,Admission Weight (Kg)
3,78716.0,112130.0,200050.0,226512.0,2149-07-14 17:56:00,72.8,Admission Weight (Kg)
4,99052.0,129142.0,200063.0,226512.0,2141-03-21 11:33:00,131.8,Admission Weight (Kg)
...,...,...,...,...,...,...,...
3077334,78288.0,109975.0,299640.0,223762.0,2131-04-17 04:00:00,36.7,Temperature Celsius
3077335,78288.0,109975.0,299640.0,223762.0,2131-04-17 06:00:00,36.8,Temperature Celsius
3077336,78288.0,109975.0,299640.0,223762.0,2131-04-17 07:00:00,36.7,Temperature Celsius
3077337,78288.0,109975.0,299640.0,223762.0,2131-04-17 08:00:00,36.8,Temperature Celsius


In [19]:
chartevents['VALUE']=chartevents['VALUE'].astype(float)
chartevents['ITEMID']=chartevents['ITEMID'].astype(int)

- **keeping only patients in the icu**

In [16]:
chartevents=chartevents[~chartevents['ICUSTAY_ID'].isna()]

In [17]:
len(chartevents)

3076571

In [20]:
pivoted_charts=chartevents.pivot_table(index=["SUBJECT_ID","HADM_ID","ICUSTAY_ID","ITEMID","CHARTTIME"],columns='LABEL',values="VALUE").reset_index()
pivoted_charts.columns.name = None
pivoted_charts

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,Admission Weight (Kg),Admit Ht,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,Arterial Blood Pressure systolic,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2,Temperature C,Temperature Celsius
0,9.0,150750.0,220597.0,211,2149-11-09 13:00:00,NaN,NaN,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,150750.0,220597.0,211,2149-11-09 13:40:00,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,150750.0,220597.0,211,2149-11-09 13:45:00,NaN,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,150750.0,220597.0,211,2149-11-09 13:50:00,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,150750.0,220597.0,211,2149-11-09 13:55:00,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069211,99955.0,108494.0,243255.0,220210,2139-07-19 13:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069212,99955.0,108494.0,243255.0,220210,2139-07-19 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069213,99955.0,108494.0,243255.0,220210,2139-07-19 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069214,99955.0,108494.0,243255.0,220210,2139-07-16 16:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN


In [21]:
pivoted_charts.drop(["ICUSTAY_ID","ITEMID"],axis=1,inplace=True)

In [22]:
df=pivoted_charts.copy()
df

,SUBJECT_ID,HADM_ID,CHARTTIME,Admission Weight (Kg),Admit Ht,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,Arterial Blood Pressure systolic,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2,Temperature C,Temperature Celsius
0,9.0,150750.0,2149-11-09 13:00:00,NaN,NaN,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,150750.0,2149-11-09 13:40:00,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,150750.0,2149-11-09 13:45:00,NaN,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,150750.0,2149-11-09 13:50:00,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,150750.0,2149-11-09 13:55:00,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069211,99955.0,108494.0,2139-07-19 13:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069212,99955.0,108494.0,2139-07-19 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069213,99955.0,108494.0,2139-07-19 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069214,99955.0,108494.0,2139-07-16 16:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN


In [23]:
#dropping ABP cols
df.drop(["Arterial Blood Pressure diastolic","Arterial Blood Pressure mean","Arterial Blood Pressure systolic"],axis=1,inplace=True)

In [24]:
df['SUBJECT_ID']=df['SUBJECT_ID'].astype(int)
df['HADM_ID']=df['HADM_ID'].astype(int)

In [27]:
df.dtypes

SUBJECT_ID                                        int32
HADM_ID                                           int32
CHARTTIME                                datetime64[ns]
Admission Weight (Kg)                           float64
Admit Ht                                        float64
Heart Rate                                      float64
Height (cm)                                     float64
Non Invasive Blood Pressure diastolic           float64
Non Invasive Blood Pressure mean                float64
Non Invasive Blood Pressure systolic            float64
Respiratory Rate                                float64
SpO2                                            float64
Temperature C                                   float64
Temperature Celsius                             float64
dtype: object

In [28]:
df.isnull().mean().round(4).mul(100).sort_values(ascending=False)

Height (cm)                              99.96
Admit Ht                                 99.96
Admission Weight (Kg)                    99.93
Temperature Celsius                      99.71
Temperature C                            99.35
Non Invasive Blood Pressure systolic     95.02
Non Invasive Blood Pressure mean         95.02
Non Invasive Blood Pressure diastolic    95.02
SpO2                                     87.61
Respiratory Rate                         74.89
Heart Rate                               74.52
CHARTTIME                                 0.00
HADM_ID                                   0.00
SUBJECT_ID                                0.00
dtype: float64

In [25]:
#checking missing rate for important vital signs
vs=["Heart Rate","Respiratory Rate","SpO2","Non Invasive Blood Pressure diastolic","Non Invasive Blood Pressure mean","Non Invasive Blood Pressure systolic"]
filtered_df=df[['SUBJECT_ID',"HADM_ID"]+vs]

In [26]:
filtered_df

,SUBJECT_ID,HADM_ID,Heart Rate,Respiratory Rate,SpO2,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic
0,9,150750,86.0,NaN,NaN,NaN,NaN,NaN
1,9,150750,85.0,NaN,NaN,NaN,NaN,NaN
2,9,150750,84.0,NaN,NaN,NaN,NaN,NaN
3,9,150750,82.0,NaN,NaN,NaN,NaN,NaN
4,9,150750,85.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3069211,99955,108494,NaN,14.0,NaN,NaN,NaN,NaN
3069212,99955,108494,NaN,14.0,NaN,NaN,NaN,NaN
3069213,99955,108494,NaN,14.0,NaN,NaN,NaN,NaN
3069214,99955,108494,NaN,14.0,NaN,NaN,NaN,NaN


In [27]:
count_df=filtered_df.groupby(["SUBJECT_ID",'HADM_ID']).count().reset_index()
count_df

,SUBJECT_ID,HADM_ID,Heart Rate,Respiratory Rate,SpO2,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic
0,9,150750,193,193,201,0,0,0
1,55,190665,57,55,22,0,0,0
2,67,155252,15,15,14,0,0,0
3,81,175016,39,39,39,0,0,0
4,83,158569,56,56,44,0,0,0
...,...,...,...,...,...,...,...,...
4524,99797,135425,113,111,0,107,106,107
4525,99814,186518,24,25,0,7,7,7
4526,99863,100749,790,783,0,269,269,269
4527,99936,107913,102,102,0,99,99,99


In [28]:
print("######## Missing Rate #############")
for col in count_df.columns[2:]:
    missing_rate=(len(count_df[count_df[col]==0])/len(count_df))*100
    print(col," ",round(missing_rate,2))

######## Missing Rate #############
Heart Rate   0.0
Respiratory Rate   0.02
SpO2   46.96
Non Invasive Blood Pressure diastolic   53.39
Non Invasive Blood Pressure mean   53.39
Non Invasive Blood Pressure systolic   53.39


In [52]:
def boolean_indexing(df):
    # Create a boolean mask for rows where all columns are greater than 0
    mask = df.iloc[:,2:].gt(0).all(axis=1)

    # Apply the boolean mask to the DataFrame
    filtered_df = df[mask]

    return filtered_df
full_df=boolean_indexing(count_df)
print("number of admissions with complete vital signs measurements: ",len(full_df))
print("number of patients with complete vital signs measurements: ",full_df['SUBJECT_ID'].nunique())
full_df

number of admissions with complete vital signs measurements:  9
number of patients with complete vital signs measurements:  9


,SUBJECT_ID,HADM_ID,Heart Rate,Respiratory Rate,SpO2,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic
1953,26851,169435,184,182,178,3,3,3
2008,27382,161290,432,431,333,33,32,33
2071,28064,148976,199,197,118,47,48,47
2194,29299,162468,356,358,112,235,233,235
2336,30648,130000,354,351,379,8,8,8
2419,31514,114047,225,223,34,47,45,47
2425,31588,178795,31,31,16,15,15,15
2471,32047,155347,366,367,139,136,138,136
2539,32701,109485,192,190,186,3,3,3


In [54]:
adm_list=full_df['HADM_ID'].unique()
final_data=df[df['HADM_ID'].isin(adm_list)]
final_data

,SUBJECT_ID,HADM_ID,CHARTTIME,Admission Weight (Kg),Admit Ht,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2,Temperature C,Temperature Celsius
1008025,26851,169435,2139-07-09 23:00:00,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008026,26851,169435,2139-07-10 00:00:00,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008027,26851,169435,2139-07-10 01:00:00,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008028,26851,169435,2139-07-10 02:00:00,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008029,26851,169435,2139-07-10 03:00:00,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271867,32701,109485,2139-04-02 18:04:00,NaN,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN
1271868,32701,109485,2139-04-02 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN
1271869,32701,109485,2139-04-02 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN
1271870,32701,109485,2139-04-02 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN


In [55]:
#filling the weight and height cols
# Fill missing values in "Height (cm)" column with any value found within each admission
final_data['Height (cm)'] = final_data.groupby('HADM_ID')['Height (cm)'].fillna(method='bfill')
final_data['Admit Ht'] = final_data.groupby('HADM_ID')['Admit Ht'].fillna(method='ffill')
final_data['Admission Weight (Kg)']=final_data.groupby('HADM_ID')['Admission Weight (Kg)'].fillna(method='ffill')

In [56]:
final_data

,SUBJECT_ID,HADM_ID,CHARTTIME,Admission Weight (Kg),Admit Ht,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2,Temperature C,Temperature Celsius
1008025,26851,169435,2139-07-09 23:00:00,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008026,26851,169435,2139-07-10 00:00:00,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008027,26851,169435,2139-07-10 01:00:00,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008028,26851,169435,2139-07-10 02:00:00,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008029,26851,169435,2139-07-10 03:00:00,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271867,32701,109485,2139-04-02 18:04:00,NaN,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN
1271868,32701,109485,2139-04-02 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN
1271869,32701,109485,2139-04-02 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN
1271870,32701,109485,2139-04-02 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN


In [58]:
final_data.isnull().mean().round(4).mul(100).sort_values(ascending=False)

Temperature Celsius                      100.00
Temperature C                            100.00
Admission Weight (Kg)                     99.87
Non Invasive Blood Pressure mean          94.30
Non Invasive Blood Pressure systolic      94.28
Non Invasive Blood Pressure diastolic     94.28
SpO2                                      83.77
Admit Ht                                  78.22
Respiratory Rate                          74.70
Heart Rate                                74.61
Height (cm)                               42.58
CHARTTIME                                  0.00
HADM_ID                                    0.00
SUBJECT_ID                                 0.00
dtype: float64

In [63]:
#dropping missing cols
final_data=final_data.drop(["Temperature C","Temperature Celsius","Admission Weight (Kg)"],axis=1)

In [76]:
final_data=final_data.drop('Admit Ht',axis=1)

In [77]:
final_data

,SUBJECT_ID,HADM_ID,CHARTTIME,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2
1008025,26851,169435,2139-07-09 23:00:00,70.0,NaN,NaN,NaN,NaN,NaN,NaN
1008026,26851,169435,2139-07-10 00:00:00,69.0,NaN,NaN,NaN,NaN,NaN,NaN
1008027,26851,169435,2139-07-10 01:00:00,73.0,NaN,NaN,NaN,NaN,NaN,NaN
1008028,26851,169435,2139-07-10 02:00:00,72.0,NaN,NaN,NaN,NaN,NaN,NaN
1008029,26851,169435,2139-07-10 03:00:00,70.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1271867,32701,109485,2139-04-02 18:04:00,NaN,NaN,NaN,84.0,NaN,NaN,NaN
1271868,32701,109485,2139-04-02 16:00:00,NaN,NaN,NaN,NaN,NaN,37.0,NaN
1271869,32701,109485,2139-04-02 17:00:00,NaN,NaN,NaN,NaN,NaN,28.0,NaN
1271870,32701,109485,2139-04-02 18:00:00,NaN,NaN,NaN,NaN,NaN,28.0,NaN


In [78]:
# Group the measurements in "charts" DataFrame by hour for each admission
charts_grouped = final_data.groupby(['HADM_ID', pd.Grouper(key='CHARTTIME', freq='H')]).mean().reset_index()

# # Count the number of observations per admission
# admission_counts = charts_grouped.groupby('HADM_ID')['HOSPITAL_EXPIRE_FLAG'].count().reset_index().rename(columns={"HOSPITAL_EXPIRE_FLAG":"count"})

# # Filter bp_data DataFrame to keep only the admissions with at least 16 hours of observations
# filtered= charts_grouped[charts_grouped['HADM_ID'].isin(admission_counts[admission_counts['count'] >= 16]['HADM_ID'].values)]

# #sorting by charttime
# filtered=filtered.sort_values('CHARTTIME')

# #getting the new number of admission
# print(filtered['HADM_ID'].nunique())

In [79]:
charts_grouped

,HADM_ID,CHARTTIME,SUBJECT_ID,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2
0,109485,2139-04-01 17:00:00,32701,96.25,NaN,NaN,NaN,NaN,24.333333,95.0
1,109485,2139-04-01 18:00:00,32701,91.00,NaN,NaN,NaN,NaN,29.000000,99.0
2,109485,2139-04-01 19:00:00,32701,91.00,NaN,NaN,NaN,NaN,25.000000,97.0
3,109485,2139-04-01 20:00:00,32701,95.00,NaN,NaN,NaN,NaN,26.000000,100.0
4,109485,2139-04-01 21:00:00,32701,91.00,NaN,NaN,NaN,NaN,17.000000,97.0
...,...,...,...,...,...,...,...,...,...,...
1901,178795,2147-09-18 12:00:00,31588,90.00,NaN,73.0,91.0,140.0,13.000000,NaN
1902,178795,2147-09-18 13:00:00,31588,88.00,NaN,77.0,84.0,106.0,11.000000,NaN
1903,178795,2147-09-18 14:00:00,31588,70.00,NaN,53.0,64.0,102.0,12.000000,NaN
1904,178795,2147-09-18 15:00:00,31588,79.00,NaN,83.0,95.0,138.0,12.000000,NaN


In [80]:
charts_grouped.isnull().mean().round(4).mul(100).sort_values(ascending=False)

Non Invasive Blood Pressure systolic     73.50
Non Invasive Blood Pressure mean         73.50
Non Invasive Blood Pressure diastolic    73.50
Height (cm)                              46.69
SpO2                                     42.44
Respiratory Rate                          1.00
Heart Rate                                0.58
SUBJECT_ID                                0.00
CHARTTIME                                 0.00
HADM_ID                                   0.00
dtype: float64

**analyzing data missingness**

In [87]:
#missing rate per feature for each admission
charts_grouped.set_index('HADM_ID').isna().groupby(level=0).mean().iloc[:,2:]

,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2
HADM_ID,,,,,,,
109485,0.000000,1.0,0.979310,0.979310,0.979310,0.006897,0.006897
114047,0.009390,0.0,0.779343,0.788732,0.779343,0.023474,0.845070
130000,0.003268,1.0,0.973856,0.973856,0.973856,0.013072,0.000000
148976,0.005587,0.0,0.737430,0.731844,0.737430,0.011173,0.480447
155347,0.006579,0.0,0.569079,0.562500,0.569079,0.006579,0.667763
161290,0.004016,1.0,0.867470,0.871486,0.867470,0.004016,0.409639
162468,0.012500,0.0,0.318750,0.318750,0.318750,0.006250,0.687500
169435,0.000000,1.0,0.981707,0.981707,0.981707,0.012195,0.012195
178795,0.000000,1.0,0.423077,0.423077,0.423077,0.000000,0.576923


In [92]:
full_df

,SUBJECT_ID,HADM_ID,Heart Rate,Respiratory Rate,SpO2,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic
1953,26851,169435,184,182,178,3,3,3
2008,27382,161290,432,431,333,33,32,33
2071,28064,148976,199,197,118,47,48,47
2194,29299,162468,356,358,112,235,233,235
2336,30648,130000,354,351,379,8,8,8
2419,31514,114047,225,223,34,47,45,47
2425,31588,178795,31,31,16,15,15,15
2471,32047,155347,366,367,139,136,138,136
2539,32701,109485,192,190,186,3,3,3


In [95]:
agg_count=charts_grouped.groupby(["SUBJECT_ID",'HADM_ID']).count().reset_index()
agg_count

,SUBJECT_ID,HADM_ID,CHARTTIME,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2
0,26851,169435,164,164,0,3,3,3,162,162
1,27382,161290,249,248,0,33,32,33,248,147
2,28064,148976,179,178,179,47,48,47,177,93
3,29299,162468,320,316,320,218,218,218,318,100
4,30648,130000,306,305,0,8,8,8,302,306
5,31514,114047,213,211,213,47,45,47,208,33
6,31588,178795,26,26,0,15,15,15,26,11
7,32047,155347,304,302,304,131,133,131,302,101
8,32701,109485,145,145,0,3,3,3,144,144


In [96]:
#admission with 10+hours
final_adm_list=agg_count[agg_count['Non Invasive Blood Pressure diastolic']>10]['HADM_ID'].to_list()
final_adm_list

[161290, 148976, 162468, 114047, 178795, 155347]

In [99]:
dataset=charts_grouped[charts_grouped['HADM_ID'].isin(final_adm_list)]
dataset

,HADM_ID,CHARTTIME,SUBJECT_ID,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2
145,114047,2175-07-31 02:00:00,31514,91.0,178.0,NaN,NaN,NaN,18.0,100.0
146,114047,2175-07-31 03:00:00,31514,94.0,178.0,NaN,NaN,NaN,18.0,100.0
147,114047,2175-07-31 04:00:00,31514,94.0,178.0,NaN,NaN,NaN,20.0,100.0
148,114047,2175-07-31 05:00:00,31514,96.0,178.0,NaN,NaN,NaN,20.0,100.0
149,114047,2175-07-31 06:00:00,31514,103.0,178.0,NaN,NaN,NaN,20.0,100.0
...,...,...,...,...,...,...,...,...,...,...
1901,178795,2147-09-18 12:00:00,31588,90.0,NaN,73.0,91.0,140.0,13.0,NaN
1902,178795,2147-09-18 13:00:00,31588,88.0,NaN,77.0,84.0,106.0,11.0,NaN
1903,178795,2147-09-18 14:00:00,31588,70.0,NaN,53.0,64.0,102.0,12.0,NaN
1904,178795,2147-09-18 15:00:00,31588,79.0,NaN,83.0,95.0,138.0,12.0,NaN


In [100]:
dataset.isnull().mean().round(4).mul(100).sort_values(ascending=False)

SpO2                                     62.43
Non Invasive Blood Pressure systolic     61.97
Non Invasive Blood Pressure mean         61.97
Non Invasive Blood Pressure diastolic    61.97
Height (cm)                              21.30
Respiratory Rate                          0.93
Heart Rate                                0.77
SUBJECT_ID                                0.00
CHARTTIME                                 0.00
HADM_ID                                   0.00
dtype: float64

- extremely small dataset (only 6 admissions) we will work with admissions that have BP vital signs measurements

In [29]:
#patients without misssing cols
count_df[(count_df['Non Invasive Blood Pressure diastolic']>0)& (count_df['Respiratory Rate']>0)& (count_df['Heart Rate']>0)]

,SUBJECT_ID,HADM_ID,Heart Rate,Respiratory Rate,SpO2,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic
16,222,103002,62,62,0,52,52,52
18,249,149546,143,142,0,93,93,93
38,502,116367,319,318,0,16,16,16
47,671,126769,198,198,0,186,185,186
77,1018,135732,49,49,0,49,49,49
...,...,...,...,...,...,...,...,...
4524,99797,135425,113,111,0,107,106,107
4525,99814,186518,24,25,0,7,7,7
4526,99863,100749,790,783,0,269,269,269
4527,99936,107913,102,102,0,99,99,99


In [32]:
bp_df=count_df[(count_df['Non Invasive Blood Pressure diastolic']>0)& (count_df['Respiratory Rate']>0)& (count_df['Heart Rate']>0)]
charts_data=df[df['HADM_ID'].isin(bp_df['HADM_ID'].unique())]
charts_data

,SUBJECT_ID,HADM_ID,CHARTTIME,Admission Weight (Kg),Admit Ht,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2,Temperature C,Temperature Celsius
5818,222,103002,2143-02-16 14:33:00,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5819,222,103002,2143-02-16 14:38:00,NaN,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5820,222,103002,2143-02-16 15:00:00,NaN,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5821,222,103002,2143-02-16 16:00:00,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5822,222,103002,2143-02-16 17:00:00,NaN,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069211,99955,108494,2139-07-19 13:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069212,99955,108494,2139-07-19 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069213,99955,108494,2139-07-19 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
3069214,99955,108494,2139-07-16 16:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN


In [33]:
agg_charts = charts_data.groupby(['HADM_ID', pd.Grouper(key='CHARTTIME', freq='H')]).mean().reset_index()
agg_charts

,HADM_ID,CHARTTIME,SUBJECT_ID,Admission Weight (Kg),Admit Ht,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2,Temperature C,Temperature Celsius
0,100018,2176-08-29 18:00:00,58128,125.0,NaN,65.5,NaN,NaN,NaN,NaN,14.00,NaN,NaN,NaN
1,100018,2176-08-29 19:00:00,58128,NaN,NaN,66.0,NaN,NaN,NaN,NaN,15.75,NaN,NaN,NaN
2,100018,2176-08-29 20:00:00,58128,NaN,NaN,70.0,NaN,NaN,NaN,NaN,15.75,NaN,NaN,NaN
3,100018,2176-08-29 21:00:00,58128,NaN,NaN,82.5,NaN,NaN,NaN,NaN,15.00,NaN,NaN,NaN
4,100018,2176-08-29 22:00:00,58128,NaN,NaN,99.0,NaN,NaN,NaN,NaN,17.50,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259504,199972,2186-09-06 08:00:00,81561,NaN,NaN,118.0,NaN,NaN,NaN,NaN,22.00,NaN,NaN,NaN
259505,199972,2186-09-06 09:00:00,81561,NaN,NaN,121.0,NaN,NaN,NaN,NaN,19.00,NaN,NaN,NaN
259506,199972,2186-09-06 10:00:00,81561,NaN,NaN,112.0,NaN,NaN,NaN,NaN,24.00,NaN,NaN,NaN
259507,199972,2186-09-06 11:00:00,81561,NaN,NaN,113.0,NaN,75.0,86.0,130.0,26.00,NaN,NaN,NaN


In [39]:
print("number of unique admission",agg_charts['HADM_ID'].nunique())

number of unique admission 2111


In [50]:
# Count the number of observations per admission
admission_counts = agg_charts.groupby('HADM_ID')['SUBJECT_ID'].count().reset_index().rename(columns={"SUBJECT_ID":"count"})

# Filter bp_data DataFrame to keep only the admissions with at least 16 hours of observations
filtered= agg_charts[agg_charts['HADM_ID'].isin(admission_counts[admission_counts['count'] >= 10]['HADM_ID'].values)]
filtered=filtered.sort_values(['HADM_ID','CHARTTIME'])
filtered

,HADM_ID,CHARTTIME,SUBJECT_ID,Admission Weight (Kg),Admit Ht,Heart Rate,Height (cm),Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,SpO2,Temperature C,Temperature Celsius
0,100018,2176-08-29 18:00:00,58128,125.0,NaN,65.5,NaN,NaN,NaN,NaN,14.00,NaN,NaN,NaN
1,100018,2176-08-29 19:00:00,58128,NaN,NaN,66.0,NaN,NaN,NaN,NaN,15.75,NaN,NaN,NaN
2,100018,2176-08-29 20:00:00,58128,NaN,NaN,70.0,NaN,NaN,NaN,NaN,15.75,NaN,NaN,NaN
3,100018,2176-08-29 21:00:00,58128,NaN,NaN,82.5,NaN,NaN,NaN,NaN,15.00,NaN,NaN,NaN
4,100018,2176-08-29 22:00:00,58128,NaN,NaN,99.0,NaN,NaN,NaN,NaN,17.50,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259504,199972,2186-09-06 08:00:00,81561,NaN,NaN,118.0,NaN,NaN,NaN,NaN,22.00,NaN,NaN,NaN
259505,199972,2186-09-06 09:00:00,81561,NaN,NaN,121.0,NaN,NaN,NaN,NaN,19.00,NaN,NaN,NaN
259506,199972,2186-09-06 10:00:00,81561,NaN,NaN,112.0,NaN,NaN,NaN,NaN,24.00,NaN,NaN,NaN
259507,199972,2186-09-06 11:00:00,81561,NaN,NaN,113.0,NaN,75.0,86.0,130.0,26.00,NaN,NaN,NaN


In [51]:
print(len(admission_counts[admission_counts['count']<10]))
admission_counts

44


,HADM_ID,count
0,100018,85
1,100037,117
2,100197,20
3,100225,26
4,100265,17
...,...,...
2106,199867,60
2107,199898,18
2108,199933,30
2109,199967,47


In [52]:
print("number of admission with more at least 10 hours of measurements: ",filtered['HADM_ID'].nunique())

number of admission with more at least 10 hours of measurements:  2067


In [54]:
filtered.isnull().mean().round(4).mul(100).sort_values(ascending=False)

Temperature C                            100.00
Admit Ht                                 100.00
SpO2                                      99.58
Height (cm)                               99.58
Admission Weight (Kg)                     99.17
Temperature Celsius                       97.53
Non Invasive Blood Pressure diastolic     44.70
Non Invasive Blood Pressure systolic      44.68
Non Invasive Blood Pressure mean          44.62
Respiratory Rate                           0.94
Heart Rate                                 0.27
SUBJECT_ID                                 0.00
CHARTTIME                                  0.00
HADM_ID                                    0.00
dtype: float64

In [55]:
filtered['Height (cm)'] = filtered.groupby('HADM_ID')['Height (cm)'].fillna(method='bfill')
filtered['Admit Ht'] = filtered.groupby('HADM_ID')['Admit Ht'].fillna(method='ffill')
filtered['Admission Weight (Kg)']=filtered.groupby('HADM_ID')['Admission Weight (Kg)'].fillna(method='ffill')

In [56]:
filtered.isnull().mean().round(4).mul(100).sort_values(ascending=False)

Temperature C                            100.00
Admit Ht                                  99.69
SpO2                                      99.58
Temperature Celsius                       97.53
Height (cm)                               79.13
Non Invasive Blood Pressure diastolic     44.70
Non Invasive Blood Pressure systolic      44.68
Non Invasive Blood Pressure mean          44.62
Admission Weight (Kg)                     15.01
Respiratory Rate                           0.94
Heart Rate                                 0.27
SUBJECT_ID                                 0.00
CHARTTIME                                  0.00
HADM_ID                                    0.00
dtype: float64

In [57]:
filtered['Height (cm)'] = filtered.groupby('HADM_ID')['Height (cm)'].fillna(method='ffill')

In [58]:
filtered.isnull().mean().round(4).mul(100).sort_values(ascending=False)

Temperature C                            100.00
Admit Ht                                  99.69
SpO2                                      99.58
Temperature Celsius                       97.53
Non Invasive Blood Pressure diastolic     44.70
Non Invasive Blood Pressure systolic      44.68
Non Invasive Blood Pressure mean          44.62
Height (cm)                               28.22
Admission Weight (Kg)                     15.01
Respiratory Rate                           0.94
Heart Rate                                 0.27
SUBJECT_ID                                 0.00
CHARTTIME                                  0.00
HADM_ID                                    0.00
dtype: float64

In [59]:
filtered.drop(['Temperature C','Admit Ht','SpO2','Temperature Celsius'],axis=1,inplace=True)

In [61]:
filtered.isnull().mean().round(4).mul(100).sort_values(ascending=False)

Non Invasive Blood Pressure diastolic    44.70
Non Invasive Blood Pressure systolic     44.68
Non Invasive Blood Pressure mean         44.62
Height (cm)                              28.22
Admission Weight (Kg)                    15.01
Respiratory Rate                          0.94
Heart Rate                                0.27
SUBJECT_ID                                0.00
CHARTTIME                                 0.00
HADM_ID                                   0.00
dtype: float64

In [64]:
filtered.to_parquet('clean_stroke_charts.parquet')